In [1]:
# Cell 1: 環境設定とセットアップ
"""
LA-Bench 2025: 実験手順生成タスク
Baseline Implementation for Google Colaboratory
GitHub: https://github.com/lasa-or-jp/la-bench.git
"""

#@title 1. 環境セットアップ { display-mode: "form" }
#@markdown このセルを実行して必要なライブラリをインストールし、リポジトリをクローンします。


import os
from pathlib import Path

# Colabかどうかの確認
try:
    import google.colab
    IN_COLAB = True
    print("✅ Google Colaboratory環境を検出しました")
    # 必要なライブラリのインストール
    print("\n📦 必要なライブラリをインストール中...")
    !pip install -q openai pyyaml tqdm pandas

    print("✅ ライブラリのインストール完了")

    # GitHubリポジトリのクローン
    REPO_URL = "https://github.com/lasa-or-jp/la-bench.git"
    REPO_NAME = "la-bench"

    if not os.path.exists(REPO_NAME):
        print(f"\n📥 リポジトリをクローン中: {REPO_URL}")
        !git clone -q {REPO_URL}
        print(f"✅ リポジトリのクローン完了: {REPO_NAME}/")
    else:
        print(f"\n📂 リポジトリは既に存在します: {REPO_NAME}/")
        print("📥 最新版に更新中...")
        !cd {REPO_NAME} && git pull -q
        print("✅ 更新完了")

    # 作業ディレクトリの設定
    WORK_DIR = Path(REPO_NAME)
    os.chdir(WORK_DIR)
    print(f"\n📍 作業ディレクトリ: {os.getcwd()}")

    # ディレクトリ構造の確認
    print("\n📊 プロジェクト構造:")
    !ls -la
except ImportError:
    IN_COLAB = False
    print("⚠️ ローカル環境で実行中です")
    if Path.cwd().name == "notebooks":
        os.chdir(Path.cwd().parent)

✅ Google Colaboratory環境を検出しました

📦 必要なライブラリをインストール中...
✅ ライブラリのインストール完了

📥 リポジトリをクローン中: https://github.com/lasa-or-jp/la-bench.git
✅ リポジトリのクローン完了: la-bench/

📍 作業ディレクトリ: /content/la-bench

📊 プロジェクト構造:
total 420
drwxr-xr-x 9 root root   4096 Nov 19 05:08 .
drwxr-xr-x 1 root root   4096 Nov 19 05:07 ..
drwxr-xr-x 2 root root   4096 Nov 19 05:08 announcements
-rw-r--r-- 1 root root   4741 Nov 19 05:08 CLAUDE.md
drwxr-xr-x 4 root root   4096 Nov 19 05:08 code
-rw-r--r-- 1 root root   2658 Nov 19 05:08 CONTRIBUTING.md
drwxr-xr-x 5 root root   4096 Nov 19 05:08 data
drwxr-xr-x 3 root root   4096 Nov 19 05:08 docs
drwxr-xr-x 8 root root   4096 Nov 19 05:08 .git
drwxr-xr-x 3 root root   4096 Nov 19 05:08 .github
-rw-r--r-- 1 root root   1086 Nov 19 05:08 .gitignore
-rw-r--r-- 1 root root   1101 Nov 19 05:08 LICENSE
drwxr-xr-x 2 root root   4096 Nov 19 05:08 notebooks
-rw-r--r-- 1 root root    569 Nov 19 05:08 pyproject.toml
-rw-r--r-- 1 root root  13396 Nov 19 05:08 README.md
-rw-r--r-- 1 root 

In [14]:
# Cell 2: OpenAI APIキーの設定
#@title 2. OpenAI API Key設定 { display-mode: "form" }
#@markdown OpenAI APIキーを入力してください。キーは安全に管理されます。


# APIキーの取得方法を選択
use_secrets = False  #@param {type:"boolean"}
#@markdown ☝️ Google Colab Secretsを使用する場合はチェック

if IN_COLAB:
    import getpass
    from google.colab import userdata
    if use_secrets:
        try:
            # Colab Secretsから取得
            API_KEY = userdata.get('OPENAI_API_KEY')
            print("✅ APIキーをSecretsから取得しました")
        except Exception as e:
            print("⚠️ Secretsからの取得に失敗しました")
            print("左側のパネルの🔑アイコンから'OPENAI_API_KEY'を設定してください")
            API_KEY = None
    else:
        # 直接入力
        api_key_input = getpass.getpass("🔑 OpenAI API Keyを入力: ")
        if api_key_input:
            API_KEY = api_key_input
            os.environ['OPENAI_API_KEY'] = API_KEY
            print("✅ APIキーが設定されました")
        else:
            API_KEY = None
            print("⚠️ APIキーが設定されていません（ヒューリスティック手法のみ使用）")
else:
    # ローカル環境の場合
    API_KEY = os.getenv("OPENAI_API_KEY")
    if not API_KEY:
        API_KEY = input("OpenAI API Key: ")

# APIキーの検証
if API_KEY:
    print(f"🔑 APIキー: {'*' * 20}{API_KEY[-4:]}")
else:
    print("⚠️ GPT機能は使用できません")

🔑 OpenAI API Keyを入力: ··········
✅ APIキーが設定されました
🔑 APIキー: ********************Ct8A


In [15]:
# Cell 3: ライブラリのインポートと設定
#@title 3. ライブラリのインポート { display-mode: "form" }

import json
import yaml
import time
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Set, Any
from pathlib import Path
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

# データ処理
import pandas as pd
from dataclasses import dataclass, field

# OpenAI API
try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False
    print("⚠️ OpenAIライブラリが利用できません")

# プログレスバー (Colab対応)
from tqdm.auto import tqdm

# ログ設定
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger(__name__)

print("="*60)
print("LA-Bench 2025 Baseline Implementation")
print(f"実行環境: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"実行時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"OpenAI利用可能: {OPENAI_AVAILABLE and API_KEY is not None}")
print("="*60)

LA-Bench 2025 Baseline Implementation
実行環境: Google Colab
実行時刻: 2025-11-19 05:16:58
OpenAI利用可能: True


In [16]:
# Cell 4: データ構造
#@title 4. データ構造の定義 { display-mode: "form" }

@dataclass
class Step:
    id: int
    text: str

@dataclass
class ReferenceEntry:
    id: int
    text: str

@dataclass
class ExampleInput:
    instruction: str
    mandatory_objects: Set[str] = field(default_factory=set)
    source_protocol_steps: List[Step] = field(default_factory=list)
    expected_final_states: Set[str] = field(default_factory=set)
    references: List[ReferenceEntry] = field(default_factory=list)

@dataclass
class ExampleOutput:
    procedure_steps: List[Step] = field(default_factory=list)

@dataclass
class Measurement:
    specific_criteria: Dict[str, int] = field(default_factory=dict)

@dataclass
class ExampleSample:
    id: str
    input: ExampleInput
    output: ExampleOutput
    measurement: Optional[Measurement] = None

def _to_set(x):
    return set(x) if isinstance(x, (list, set, tuple)) else set()

def _to_list(x):
    return list(x) if isinstance(x, (list, set, tuple)) else (x if isinstance(x, list) else [])

def _to_steps(x) -> List[Step]:
    steps: List[Step] = []
    arr = _to_list(x)
    if not arr:
        return steps
    if isinstance(arr[0], dict):
        for it in arr:
            try:
                sid = int(it.get("id", len(steps) + 1))
            except Exception:
                sid = len(steps) + 1
            steps.append(Step(id=sid, text=str(it.get("text", "")).strip()))
    else:
        for idx, s in enumerate(arr, start=1):
            steps.append(Step(id=idx, text=str(s).strip()))
    return steps

def _to_references(x) -> List[ReferenceEntry]:
    refs: List[ReferenceEntry] = []
    arr = _to_list(x)
    if not arr:
        return refs
    if isinstance(arr[0], dict):
        for it in arr:
            try:
                rid = int(it.get("id", len(refs) + 1))
            except Exception:
                rid = len(refs) + 1
            refs.append(ReferenceEntry(id=rid, text=str(it.get("text", "")).strip()))
    else:
        for idx, ref in enumerate(arr, start=1):
            refs.append(ReferenceEntry(id=idx, text=str(ref).strip()))
    return refs

def parse_sample(obj: Dict[str, Any]) -> ExampleSample:
    sid = obj.get("id") or obj.get("sample_id") or "unknown"
    i = obj.get("input", {})
    o = obj.get("output", {})
    m = obj.get("measurement", {})

    # Measurement.specific_criteria を dict に正規化（list形式も許容）
    sc_raw = m.get("specific_criteria", {})
    sc: Dict[str, int] = {}
    if isinstance(sc_raw, dict):
        for k, v in sc_raw.items():
            try:
                sc[str(k)] = int(v)
            except Exception:
                pass
    elif isinstance(sc_raw, list):
        for it in sc_raw:
            try:
                k = it.get("item")
                v = int(it.get("score", 0))
                if k:
                    sc[str(k)] = v
            except Exception:
                pass

    sample = ExampleSample(
        id=str(sid),
        input=ExampleInput(
            instruction=str(i.get("instruction", "")).strip(),
            mandatory_objects=_to_set(i.get("mandatory_objects", [])),
            source_protocol_steps=_to_steps(i.get("source_protocol_steps", [])),
            expected_final_states=_to_set(i.get("expected_final_states", [])),
            references=_to_references(i.get("references", [])),
        ),
        output=ExampleOutput(
            procedure_steps=_to_steps(o.get("procedure_steps", []))
        ),
        measurement=Measurement(specific_criteria=sc) if sc else None
    )
    return sample

def load_example_jsonl(path: str):
    samples = []
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"JSONL not found: {p}")
    for line in p.read_text(encoding="utf-8").splitlines():
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
        except Exception as e:
            print(f"⚠️ JSONL parse error: {e}")
            continue
        samples.append(parse_sample(obj))
    return samples

In [17]:
# Cell 5: JSONLローダーの利用
#@title 5. JSONLファイルを読み込む { display-mode: "form" }
#@markdown exampleを使うとき：`data/example/example.jsonl`
#@markdown public_testを使うとき：`data/public_test/public_test.jsonl`

jsonl_path = 'data/public_test/public_test.jsonl'  #@param {type:'string'}

try:
    samples = load_example_jsonl(jsonl_path)
    print(f'✅ Loaded {len(samples)} samples from {jsonl_path}')
except Exception as e:
    print(f'❌ Load error: {e}')


✅ Loaded 10 samples from data/public_test/public_test.jsonl


In [21]:
# Cell 6: 実験手順の生成（OpenAI, Pydantic構造化）
#@title 6. LLMで Input から Output（procedure_steps）を生成 { display-mode: "form" }

from pydantic import BaseModel, Field

# モデル設定
MODEL_NAME = "gpt-4o-2024-08-06" #@param ["gpt-4.1-mini-2025-04-14", "gpt-4o-2024-08-06", "gpt-5-2025-08-07", "gpt-5-mini-2025-08-07", "gpt-5-nano-2025-08-07"]
#@markdown gpt-4o-mini, gpt-4o-2024-08-06, あるいはそれ以降のモデルに対応しています。<br>
#@markdown (Structured outputを使用しているため) <br>
#@markdown gpt-5系モデルを使用する場合、temperature=1.0としてください。
TEMPERATURE = 0.7 # @param

#@markdown `build_messages`関数において、LLMの入力を設計しています。

class StepModel(BaseModel):
    id: int = Field(ge=1, description="ステップ番号")
    text: str = Field(description="実験手順の詳細な説明")

class GeneratedOutput(BaseModel):
    procedure_steps: List[StepModel] = Field(
        description="実験手順のリスト",
        min_items=1,
        max_items=50
    )

def build_messages(sample: ExampleSample) -> list[dict]:
    sys = (
        "あなたは、化学、生物学、および材料科学を含む全分野の**ラボオートメーション (LA) プロトコル変換の最高専門家**です。"
        "あなたの唯一の任務は、提供された Input を読み込み、LAシステムが実行可能な**極めて厳格で詳細な実験手順**（procedure_steps）を返すことです。"
        "最重要制約: 出力は、Pydanticスキーマに厳格に準拠した**JSONオブジェクトのみ**であり、いかなる説明やコードブロックも許されません。"
        "制約: ステップ数は最大50、各ステップは10文以下、idは1から昇順でなければなりません。"
    )
    user_lines = []
    user_lines.append(f"【実験指示】\n{sample.input.instruction}")
    if sample.input.mandatory_objects:
        user_lines.append("\n【使用する物品】")
        for it in sorted(sample.input.mandatory_objects):
            user_lines.append(f"- {it}")
    if sample.input.source_protocol_steps:
        user_lines.append("\n【元プロトコルの手順（参考）】")
        for st in sample.input.source_protocol_steps:
            user_lines.append(f"- {st.id}. {st.text}")
    if sample.input.expected_final_states:
        user_lines.append("\n【期待される最終状態】")
        for fs in sorted(sample.input.expected_final_states):
            user_lines.append(f"- {fs}")
    if sample.input.references:
        user_lines.append("\n【参考文献】")
        for ref in sample.input.references:
            user_lines.append(f"- [{ref.id}] {ref.text}")
    usr = "\n".join(user_lines)
    return [
        {"role": "system", "content": sys},
        {"role": "user", "content": usr},
    ]

def generate_outputs(samples: list[ExampleSample]) -> list[dict]:
    client = OpenAI(api_key=API_KEY)
    results: list[dict] = []
    for sm in samples:
        msgs = build_messages(sm)
        try:
            completion = client.chat.completions.parse(
                model=MODEL_NAME,
                messages=msgs,
                temperature=TEMPERATURE,
                response_format=GeneratedOutput,
            )
            parsed: GeneratedOutput = completion.choices[0].message.parsed  # type: ignore
            steps = [
                Step(id=s.id, text=s.text)
                for s in sorted(parsed.procedure_steps, key=lambda x: x.id)
            ][:50]
        except Exception as e:
            print(f"❌ 生成失敗: {sm.id}: {e}")
            steps = []  # no fallback
        results.append({
            "id": sm.id,
            "procedure_steps": [{"id": s.id, "text": s.text} for s in steps],
        })
    print(f"✅ 生成完了: {len(results)} samples")
    return results

# 実行
generated_results = generate_outputs(samples)
if generated_results:
    print(f"例: {generated_results[0]['id']} → {len(generated_results[0]['procedure_steps'])} steps")

# 生成結果を JSONL で保存し、ダウンロードリンクを表示
ts = time.strftime('%Y%m%d_%H%M%S')
out_dir = Path('./outputs/runs')
out_dir.mkdir(parents=True, exist_ok=True)
jsonl_path = out_dir / f'generated_{ts}.jsonl'
with jsonl_path.open('w', encoding='utf-8') as f:
    for rec in generated_results:
        obj = {"id": rec["id"], "output": {"procedure_steps": rec["procedure_steps"]}}
        line = json.dumps(obj, ensure_ascii=False, separators=(",", ":"))
        f.write(line + "\n")
print(f"📄 Saved JSONL: {jsonl_path}")

# ダウンロード（Colab/ローカル双方に対応）
try:
    from google.colab import files as colab_files  # type: ignore
    # ダウンロード確認ダイアログを出して、yならダウンロード
    from google.colab.output import eval_js
    print(f"Download file: {jsonl_path}")
    confirm = eval_js('confirm("生成されたJSONLファイルをダウンロードしますか？")')
    if confirm:
      colab_files.download(str(jsonl_path))
    else:
      print("ダウンロードをスキップしました。")

except Exception:
    from IPython.display import FileLink, display
    display(FileLink(str(jsonl_path.resolve())))

✅ 生成完了: 10 samples
例: public_test_1 → 16 steps
📄 Saved JSONL: outputs/runs/generated_20251119_054439.jsonl
Download file: outputs/runs/generated_20251119_054439.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Cell 7: LLM-as-a-judge 評価（10点満点）
#@title 7. LLM で共通5点 + 個別5点を採点 { display-mode: "form" }

import time
import pandas as pd
from typing import List, Optional
from pydantic import BaseModel, Field

try:
    from openai import OpenAI
except Exception as e:
    raise RuntimeError("OpenAI SDK v1 が見つかりません。`uv add openai` で追加してください。") from e

JUDGE_MODEL = "gpt-4.1-mini"  # 高性能推奨モデルに変更可
JUDGE_TEMPERATURE = 0.2

class JudgeOutput(BaseModel):
    general_score: float = Field(ge=0, le=5)
    specific_score: float = Field(ge=0, le=5)
    final_score: float = Field(ge=0, le=10)
    general_reason: str
    specific_matches: List[str] = []
    notes: Optional[str] = None

def build_judge_messages(sample: ExampleSample, steps: List[Step]) -> list[dict]:
    # 評価基準（共通5点 + 個別5点）
    system = (
        "あなたは生命科学実験の専門家であり、公平な採点者です。"
        "以下の基準に従って、与えられた Input と生成手順（Output）を評価し、"
        "general_score(0-5) と specific_score(0-5) と final_score(0-10) を出力してください。"
        "\n\n[共通採点基準 5点満点]\n"
        "加点(+1ずつ): 1) 実験指示のパラメータ反映, 2) 使用する物品の反映, 3) 元手順の論理反映, 4) 期待される最終状態の達成, 5) 適切な補完。\n"
        "減点: 不自然な日本語/ハルシネーション, 計算ミス, 手順矛盾。\n"
        "上限: 入力手順の丸写し等の過度の安全性が見られる場合、general_score は最大2点に制限。\n\n"
        "[個別採点基準 5点満点]\n"
        "与えられた specific_criteria の各 item が手順に含まれる/満たすなら、その score を加点（合計5点で上限）。"
    )

    parts = []
    parts.append(f"【実験指示】\n{sample.input.instruction}")
    if sample.input.mandatory_objects:
        parts.append("\n【使用する物品】")
        for it in sorted(sample.input.mandatory_objects):
            parts.append(f"- {it}")
    if sample.input.source_protocol_steps:
        parts.append("\n【元プロトコルの手順（参考）】")
        for st in sample.input.source_protocol_steps:
            parts.append(f"- {st.id}. {st.text}")
    if sample.input.expected_final_states:
        parts.append("\n【期待される最終状態】")
        for fs in sorted(sample.input.expected_final_states):
            parts.append(f"- {fs}")
    if sample.input.references:
        parts.append("\n【参考文献】")
        for ref in sample.input.references:
            parts.append(f"- [{ref.id}] {ref.text}")

    parts.append("\n【生成手順（Output）】")
    for s in steps:
        parts.append(f"- {s.id}. {s.text}")

    parts.append("\n【specific_criteria】")
    if sample.measurement and sample.measurement.specific_criteria:
        for item, sc in sample.measurement.specific_criteria.items():
            parts.append(f"- ({int(sc)}点) {item}")
    else:
        parts.append("- なし")

    user = "\n".join(parts)
    return [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]

def judge_with_llm(samples: List[ExampleSample], generated: list[dict]) -> pd.DataFrame:
    client = OpenAI(api_key=API_KEY) if 'API_KEY' in globals() and API_KEY else OpenAI()
    proc_map = {g['id']: [Step(id=it['id'], text=it['text']) for it in g['procedure_steps']] for g in generated}
    rows = []
    quota_exhausted = False
    def _is_insufficient_quota(err: Exception) -> bool:
        s = str(err)
        return 'insufficient_quota' in s or 'You exceeded your current quota' in s
    for sm in samples:
        if quota_exhausted:
            print(f"⏭️ スキップ採点: {sm.id}（クォータ不足）")
            rows.append({
                'id': sm.id,
                'general_score': 0.0,
                'specific_score': 0.0,
                'total_score': 0.0,
                'notes': 'skipped_due_to_quota',
            })
            continue
        steps = proc_map.get(sm.id, [])
        msgs = build_judge_messages(sm, steps)
        try:
            completion = client.chat.completions.parse(
                model=JUDGE_MODEL,
                messages=msgs,
                temperature=JUDGE_TEMPERATURE,
                response_format=JudgeOutput,
            )
            parsed: JudgeOutput = completion.choices[0].message.parsed  # type: ignore
            rows.append({
                'id': sm.id,
                'general_score': parsed.general_score,
                'specific_score': parsed.specific_score,
                'total_score': parsed.final_score,
                'notes': parsed.notes or '',
            })
        except Exception as e:
            print(f"❌ 評価失敗: {sm.id}: {e}")
            if _is_insufficient_quota(e):
                print("⚠️ APIクォータ不足のため、以降の採点を中断します。プラン/課金設定をご確認ください。")
                quota_exhausted = True
            rows.append({
                'id': sm.id,
                'general_score': 0.0,
                'specific_score': 0.0,
                'total_score': 0.0,
                'notes': 'evaluation_failed',
            })
    return pd.DataFrame(rows)

# 実行
df = judge_with_llm(samples, generated_results)
print(f"✅ LLM-as-a-judge: Scored {len(df)} samples (0-10)")
try:
    display(df[['id','general_score','specific_score','total_score']])
except Exception:
    print(df[['id','general_score','specific_score','total_score']])

csv_path = out_dir / f'eval_llm_{ts}.csv'
df.to_csv(csv_path, index=False, encoding="utf_8_sig")
print(f'📄 Saved: {csv_path}')

try:
    # ダウンロード確認ダイアログを出して、yならダウンロード
    print(f"Download file: {csv_path}")
    confirm = eval_js('confirm("生成されたCSVファイルをダウンロードしますか？")')
    if confirm:
      colab_files.download(str(csv_path))
    else:
      print("ダウンロードをスキップしました。")

except Exception:
    display(FileLink(str(csv_path.resolve())))


17:47:06 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:47:10 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:47:15 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:47:19 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:47:26 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✅ LLM-as-a-judge: Scored 5 samples (0-10)


,id,general_score,specific_score,total_score
0,sample_1,5.0,3.0,8.0
1,sample_2,5.0,3.0,8.0
2,sample_3,5.0,0.0,5.0
3,sample_4,5.0,3.0,8.0
4,sample_5,5.0,4.0,9.0


📄 Saved: outputs/runs/eval_llm_20251011_174658.csv
Download file: outputs/runs/eval_llm_20251011_174658.csv


/Users/kato/Dropbox/Programs/gensurv/la-bench/outputs/runs/eval_llm_20251011_174658.csv